In [1]:
import sys
sys.path.insert(0,'/home/csunix/schtmt/NewFolder/caffe_Sep_anaconda/python')
import numpy as np
import matplotlib.pyplot as plt
import caffe
import h5py
import cv2
import fnmatch
import os
from PIL import Image


caffe.set_device(0)
caffe.set_mode_gpu()


In [2]:
net = caffe.Net('test_input5frames.prototxt',
                '../convLSTM_models/ped2_aug1_epoch80.caffemodel', 
                caffe.TEST) 

In [4]:
gt_location = '/usr/not-backed-up/1_DATABASE/UCSD_Anomaly_Dataset.tar/UCSD_Anomaly_Dataset.v1p2/VideoSequences/UCSDped2/Test_gt_location'
frame_path='/usr/not-backed-up/1_DATABASE/UCSD_Anomaly_Dataset.tar/UCSD_Anomaly_Dataset.v1p2/UCSDped2/Test/'
for seq in range(1,2): # bicycle
# for seq in range(3,4): # car
# for seq in range(7,8): # 
# for seq in range(5,6): # 
# -----------------------------------------------------------
    print('sequence:',seq+1)    
    # load images
    if seq < 9:                
        dirpath = (frame_path + 'Test00%d' % (seq+1))
    else:
        dirpath = (frame_path + 'Test0%d' % (seq+1))
    nfr = len(fnmatch.filter(os.listdir(dirpath), '*.tif'))
#         frame_seq = np.zeros((227,227,nfr),dtype=np.float64)
    for fr in range(nfr):            
        if fr < 9:
            fr_id = '00%d' % (fr+1)                                       
        elif fr < 99:
            fr_id = '0%d' % (fr+1)                
        else:
            fr_id = '%d' % (fr+1)                
        im1 = Image.open(dirpath + '/' + fr_id + '.tif')
        im1 = np.array(im1.resize((227,227),Image.BILINEAR))#size [width,height]            
        im1 = np.reshape(im1,[1,227,227])
        im1 = im1.astype(float) / 255.
        if fr == 0:
            frame_seq = im1
        else:
            frame_seq = np.append(frame_seq,im1,0)
    frame_error = np.zeros([1,nfr-9])
    print(frame_error.shape)
##################################################
    test_fr = 151
#     for fr in range(105,106): # (1,2)
#     for fr in range(130,131): # (3,4)
    for fr in range(test_fr,test_fr+1): 
        cur_vol = frame_seq[fr:fr+10,:,:]        
        net.blobs['input'].data[...] = np.reshape(cur_vol[0:5,:,:],[1,5,1,227,227])
        out = net.forward()

('sequence:', 2)
(1, 171)


In [5]:
# prediction
pred = net.blobs['deconv3'].data
pred_gt = cur_vol[5:10,:,:]
pred = np.reshape(pred,[5,227,227])
error = np.power(pred_gt - pred,2)
err = np.reshape(error,[5,227*227])
err = np.sqrt(np.sum(err,1))
print(np.sum(err))

41.0323103669


In [6]:
plt.figure(figsize=(100,100))
for i in range(5):
#     print(i)
    plt.subplot(2,5,i+1)
    input_ = np.reshape(pred[i,:,:],[227,227]) 
    plt.imshow(input_,cmap='gray')
    plt.axis('off') 
for i in range(5):
    plt.subplot(2,5,i+6)
    input_ = np.reshape(error[i,:,:],[227,227])
#     plt.imshow(input_)
    imgplot = plt.imshow(input_,clim=(0.0,0.2))
    imgplot.set_cmap('jet')
    plt.axis('off') 
plt.subplots_adjust(bottom=0.565,wspace=0.01,hspace=0.00000000001)#for 2 rows
# plt.subplots_adjust(bottom=0.405,wspace=0.01,hspace=0.00000000001)#for 3 rows
# plt.tight_layout()
# plt.subplots_adjust(wspace=0.01,hspace=0.9)
# plt.show()
plt.savefig('Ped2_biker_prediction6_seq2fr156-160_error41p03.jpg')


In [7]:
plt.figure(figsize=(100,100))
for i in range(5):
    plt.subplot(1,5,i+1)    
    input_ = np.reshape(pred_gt[i,:,:],[227,227])
    plt.imshow(input_,cmap='gray')
    plt.axis('off')
plt.subplots_adjust(bottom=0.565,wspace=0.01,hspace=0.00000000001)#for 2 rows
# plt.show()    
plt.savefig('Ped2_biker_targetSeq_seq2fr156-160.jpg')

In [ ]:
#reconstruction
recons = net.blobs['deconv3'].data
prediction = cur_vol[:,5:10,:,:,:]
prediction = np.reshape(prediction,[5,1,227,227])
prediction = np.reshape(prediction,[5,227,227])
recons = np.reshape(recons,[5,227,227])
error = np.power(prediction - recons,2)
err = np.reshape(error,[5,227*227])
err = np.sqrt(np.sum(err,1))
print(np.sum(err))

# for visualization
with h5py.File(os.path.join(gt_location,'Test%d_gt_location.mat'%(seq+1))) as f:    
    data = [f[element[0]][:] for element in f['gt_location']]
# data_ = np.array(data)

import matplotlib.patches as patches
#####################

fig2 = plt.figure(figsize=(100,100))
# plt.figure(figsize=(100,100))
for i in range(5):
    plt.subplot(3,5,i+1)
    #     input_ = np.reshape(prediction[i,:,:,:,:],[227,227])
    input_ = np.reshape(prediction[i,:,:],[227,227])
    plt.imshow(input_,cmap='gray')
    plt.axis('off')
for i in range(5):
#     print(i)
    plt.subplot(3,5,i+6)
    input_ = np.reshape(recons[i,:,:],[227,227]) 
    plt.imshow(input_,cmap='gray')
    plt.axis('off') 
# for i in range(5):
#     plt.subplot(3,5,i+11)
#     input_ = np.reshape(error[i,:,:],[227,227])
# #     plt.imshow(input_)
#     imgplot = plt.imshow(input_,clim=(0.0,0.2))    
#     imgplot.set_cmap('jet')
#     plt.axis('off') 
# plt.subplots_adjust(bottom=0.405,wspace=0.01,hspace=0.00000000001)
# # plt.tight_layout()
# # plt.subplots_adjust(wspace=0.01,hspace=0.9)
# plt.show()
# # plt.savefig('bike_ped2_recons.jpg')


ax1 = fig2.add_subplot(3,5,11)
input_ = np.reshape(error[0,:,:],[227,227])
ax1.imshow(input_,clim=(0.0,0.2),cmap='jet') 
# gt_fr = np.transpose(data_[fr+5],[1,0])
data_ = np.array(data[fr+5])
gt_fr = np.transpose(data_,[1,0])
for id in range(gt_fr.shape[0]):
    x = gt_fr[id,0]
    y = gt_fr[id,1]
    w = gt_fr[id,2]
    h = gt_fr[id,3]
    rect = patches.Rectangle((x,y),w,h,linewidth=5,edgecolor='y',facecolor='none')
    ax1.add_patch(rect)

##############
ax2 = fig2.add_subplot(3,5,12)
input_ = np.reshape(error[1,:,:],[227,227])
ax2.imshow(input_,clim=(0.0,0.2),cmap='jet') 
# gt_fr = np.transpose(data_[fr+6],[1,0])
data_ = np.array(data[fr+6])
gt_fr = np.transpose(data_,[1,0])
for id in range(gt_fr.shape[0]):
    x = gt_fr[id,0]
    y = gt_fr[id,1]
    w = gt_fr[id,2]
    h = gt_fr[id,3]
    rect = patches.Rectangle((x,y),w,h,linewidth=5,edgecolor='y',facecolor='none')
    ax2.add_patch(rect)
#############
##############
ax3 = fig2.add_subplot(3,5,13)
input_ = np.reshape(error[2,:,:],[227,227])
ax3.imshow(input_,clim=(0.0,0.2),cmap='jet') 
# gt_fr = np.transpose(data_[fr+7],[1,0])
data_ = np.array(data[fr+7])
gt_fr = np.transpose(data_,[1,0])
for id in range(gt_fr.shape[0]):
    x = gt_fr[id,0]
    y = gt_fr[id,1]
    w = gt_fr[id,2]
    h = gt_fr[id,3]
    rect = patches.Rectangle((x,y),w,h,linewidth=5,edgecolor='y',facecolor='none')
    ax3.add_patch(rect)
#############
##############
ax4 = fig2.add_subplot(3,5,14)
input_ = np.reshape(error[3,:,:],[227,227])
ax4.imshow(input_,clim=(0.0,0.2),cmap='jet') 
# gt_fr = np.transpose(data_[fr+8],[1,0])
data_ = np.array(data[fr+8])
gt_fr = np.transpose(data_,[1,0])
for id in range(gt_fr.shape[0]):
    x = gt_fr[id,0]
    y = gt_fr[id,1]
    w = gt_fr[id,2]
    h = gt_fr[id,3]
    rect = patches.Rectangle((x,y),w,h,linewidth=5,edgecolor='y',facecolor='none')
    ax4.add_patch(rect)
#############
##############
ax5 = fig2.add_subplot(3,5,15)
input_ = np.reshape(error[4,:,:],[227,227])
ax5.imshow(input_,clim=(0.0,0.2),cmap='jet') 
# gt_fr = np.transpose(data_[fr+9],[1,0])
data_ = np.array(data[fr+9])
gt_fr = np.transpose(data_,[1,0])
for id in range(gt_fr.shape[0]):
    x = gt_fr[id,0]
    y = gt_fr[id,1]
    w = gt_fr[id,2]
    h = gt_fr[id,3]
    rect = patches.Rectangle((x,y),w,h,linewidth=5,edgecolor='y',facecolor='none')
    ax5.add_patch(rect)
#############
plt.subplots_adjust(bottom=0.405,wspace=0.01,hspace=0.00000000001)
plt.show()

# plt.savefig('biker_seq9ped2_frame100_predict.jpg')

In [ ]:


# fig = plt.figure(figsize=(25,25))
# plt.subplot(1,3,1)    
# input_ = np.reshape(prediction[2,:,:],[227,227])
# plt.imshow(input_,cmap='gray')
# plt.axis('off')


# plt.subplot(1,3,2)
# input_ = np.reshape(recons[2,:,:],[227,227]) 
# plt.imshow(input_,cmap='gray')
# plt.axis('off') 


# a = np.sum(error,0)
# input_ = np.reshape(a,[227,227])/5
# ax = fig.add_subplot(1,3,3)

# ax.imshow(input_,clim=(0.0,0.1),cmap='jet') 
# data_ = np.array(data[fr+9])
# gt_fr = np.transpose(data_,[1,0])
# for id in range(gt_fr.shape[0]):
#     x = gt_fr[id,0]
#     y = gt_fr[id,1]
#     w = gt_fr[id,2]
#     h = gt_fr[id,3]
#     rect = patches.Rectangle((x,y),w,h,linewidth=5,edgecolor='y',facecolor='none')
#     ax.add_patch(rect)
#     ax.axis('off')
#     plt.subplots_adjust(bottom=0.405,wspace=0.01,hspace=0.00000000001)
# plt.show()    
# # plt.savefig('biker_seq3ped2_frame80_predict.jpg')